<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/04_Apartment_buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию

## 4. Многоквартирные дома. Аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами.

### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).

In [1]:
initiate = True
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection

Mounted at /content/drive
/content/anomaly-detection


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "anomaly_detection"))

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from data.make_dataset import load_data
from features.period_anomalies import anomaly_detection_pipeline, select_anomalies

In [4]:
import warnings
warnings.filterwarnings('ignore')

## 1. Чтение данных

In [5]:
%%time
save = True
data, temperature, buildings = load_data(save=save)

100%|██████████| 24/24 [00:09<00:00,  2.48it/s]


CPU times: user 10.8 s, sys: 240 ms, total: 11.1 s
Wall time: 11.7 s


## 2. Оценка аномалий

In [6]:
%%time
ALPHA = 5
BETA = 95

(period_results, all_periods_anomalies, all_periods_anomalies_pivot) = (
    anomaly_detection_pipeline(
        data, temperature, buildings, alpha=ALPHA, beta=BETA
    )
)

100%|██████████| 24/24 [00:39<00:00,  1.63s/it]


CPU times: user 42.9 s, sys: 902 ms, total: 43.8 s
Wall time: 44.6 s


In [7]:
print(f"all_periods_anomalies.shape = {all_periods_anomalies.shape}\n")
all_periods_anomalies.sample(2)

all_periods_anomalies.shape = (1271, 17)



,Адрес объекта 2,Тип объекта,№ ОДПУ,Вид энерг-а ГВС,Этажность объекта,Дата постройки,Общая площадь объекта,Группа этажность объекта,Группа год постройки,"Текущее потребление, Гкал",Удельное потребление теплоэнергии на кв.метр площади,Hotelling's T-squared,Q residuals,ниже медианы,25% ниже медианы,25% выше медианы,Период потребления
1626,"г Уфа, ул. Комсомольская, д.27",Многоквартирный дом,1500240,1,9.0,1991-01-01,10497.0,5-9 этажей,1990-2000 гг.,302.973,0.028863,1.084291,3.042409,False,False,True,2021-10-01
105,"г Уфа, б-р. Тухвата Янаби, д.73",Многоквартирный дом,69203,0,2.0,1960-01-01,563.7,1-2 этажа,1959-1989 гг.,10.950,0.019425,0.305115,3.253450,True,True,False,2021-12-01


In [8]:
print(f"all_periods_anomalies_pivot.shape = {all_periods_anomalies_pivot.shape}\n")
all_periods_anomalies_pivot.sample(2)

all_periods_anomalies_pivot.shape = (358, 33)



,Адрес объекта 2,Тип объекта,№ ОДПУ,Вид энерг-а ГВС,Этажность объекта,Дата постройки,Общая площадь объекта,Группа этажность объекта,Группа год постройки,2021-07-01 00:00:00,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1704,"г Уфа, ул. Ленина, д.150 корп.1",Многоквартирный дом,33263,1,6.0,2006-01-01,4018.6,5-9 этажей,2001-2010 гг.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2393,"г Уфа, ул. Революционная, д.54",Многоквартирный дом,110760,1,5.0,1969-01-01,4597.1,5-9 этажей,1959-1989 гг.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
n = 0
print("Число аномалий за период 25% ниже/ выше медианного значения по группе, всего:")
for i in range(24):
    result = period_results[i]
    t = result.columns[9]

    alpha = ALPHA
    beta = BETA
    threshold = 0.25

    cond1, cond2, cond3, cond4, cond5 = select_anomalies(result, alpha, beta, threshold)
    below_median = result[(cond1 | cond2) & cond4]
    above_median = result[(cond1 | cond2) & cond5]

    n_below_median = below_median.shape[0]
    n_above_median = above_median.shape[0]
    total = n_below_median + n_above_median
    n += total

    print(f"{t.strftime('%Y-%m')}: {n_below_median} {n_above_median}\t{total}")
print(f"Аномалий всего: {n}")

Число аномалий за период 25% ниже/ выше медианного значения по группе, всего:
2021-07: 12 20	32
2021-08: 2 45	47
2021-09: 10 28	38
2021-10: 32 62	94
2021-11: 27 36	63
2021-12: 27 27	54
2022-01: 30 27	57
2022-02: 24 24	48
2022-03: 27 29	56
2022-04: 32 41	73
2022-05: 38 7	45
2022-06: 4 51	55
2022-07: 17 18	35
2022-08: 11 18	29
2022-09: 11 17	28
2022-10: 39 76	115
2022-11: 23 36	59
2022-12: 26 31	57
2023-01: 29 22	51
2023-02: 24 20	44
2023-03: 29 23	52
2023-04: 40 51	91
2023-05: 6 16	22
2023-06: 8 18	26
Аномалий всего: 1271
